In [1]:
# Import required libraries
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from config import db_config
from plotly.subplots import make_subplots
import warnings
import ipywidgets as widgets
from IPython.display import display
from dash import Dash, html, dcc, Input, Output
import dash_bootstrap_components as dbc
from sshtunnel import SSHTunnelForwarder


# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
warnings.filterwarnings('ignore')

# Set plotting style (using default instead of seaborn-v0_8 to avoid crash)
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


c:\Users\evan\.pyenv\pyenv-win\versions\3.11.5\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\evan\.pyenv\pyenv-win\versions\3.11.5\Lib\site-packages\paramiko\transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
# Simple direct connection (no tunnel needed)
connection = mysql.connector.connect(
    host=db_config["host"],
    port=db_config["port"],
    user=db_config["user"],
    password=db_config["password"],
    database=db_config["database"]
)
cursor = connection.cursor()
print("Database connection established")

Database connection established


In [3]:
teams_dict = {
    'team_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 46, 47, 48, 49, 52, 53, 54, 55, 56, 59, 68],
    'team_name': ['New Jersey Devils', 'New York Islanders', 'New York Rangers', 'Philadelphia Flyers', 'Pittsburgh Penguins', 'Boston Bruins', 'Buffalo Sabres', 'Montreal Canadiens', 'Ottawa Senators', 'Toronto Maple Leafs', 'Atlanta Thrashers', 'Carolina Hurricanes', 'Florida Panthers', 'Tampa Bay Lightning', 'Washington Capitals', 'Chicago Blackhawks', 'Detroit Red Wings', 'Nashville Predators', 'St Louis Blues', 'Calgary Flames', 'Colorado Avalanche', 'Edmonton Oilers', 'Vancouver Canucks', 'Anaheim Ducks', 'Dallas Stars', 'Los Angeles Kings', 'Pheonix Coyotes', 'San Jose Sharks', 'Columbus Blue Jackets', 'Minnesota Wild', 'Minnesota North Stars', 'Quebec Nordique', 'Winnipeg Jets (1979)', 'Hartford Whalers', 'Colorado Rockies', 'Oakland Seals', 'Atlanta Flames', 'Kansas City Scouts', 'Cleveland Barons', 'Winnipeg Jets', 'Arizona Coyotes', 'Vegas Golden Knights', 'Seattle Kraken', 'California Golden Seals', 'Utah Hockey Club', 'Utah Mammoth'],
    'team_abbreviation': ['NJD', 'NYI', 'NYR', 'PHI', 'PIT', 'BOS', 'BUF', 'MTL', 'OTT', 'TOR', 'ATL', 'CAR', 'FLA', 'TBL', 'WSH', 'CHI', 'DET', 'NSH', 'STL', 'CGY', 'COL', 'EDM', 'VAN', 'ANA', 'DAL', 'LAK', 'PHX', 'SJS', 'CBJ', 'MIN', 'MNS', 'QUE', 'WIN', 'HFD', 'CLR', 'OAK', 'AFM', 'KCS', 'CLE', 'WPG', 'ARI', 'VGK', 'SEA', 'CGS', 'UTA', 'UTA']
}

In [4]:
cursor.execute('''
    SELECT * FROM games
    WHERE season_id = 20242025
''')

games = cursor.fetchall()

#for each game_id in games, get all events
game_ids = [game[0] for game in games]

cursor.execute('''
    SELECT * FROM events
    WHERE game_id IN (%s)
    AND type_desc_key IN ('shot-on-goal', 'goal', 'missed-shot', 'failed-shot-attempt')
''' % ','.join(map(str, game_ids)))

events = cursor.fetchall()

headers = [col[0] for col in cursor.description]    # column names
events_df = pd.DataFrame(events, columns=headers)

print(events_df[['x_coord', 'y_coord']].describe())

             x_coord        y_coord
count  128085.000000  128085.000000
mean       -0.278791      -0.429699
std        64.251986      20.244438
min       -99.000000     -42.000000
25%       -65.000000     -15.000000
50%        -1.000000       0.000000
75%        64.000000      14.000000
max        99.000000      42.000000


In [ ]:
#adjust x_coord and y_coord to be positive
events_df['x_coord'] = events_df['x_coord'].abs()
